In [35]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle

In [36]:
%run ./load_code.ipynb

In [37]:
new_dic = {'cores':{},'tiepoints':{},'metadata':{}}

In [38]:
## new version: scan entire folder

datafolder = './data/coredata/'
p = Path(datafolder)  
files = sorted(list(p.glob(f'*.xlsx')))
for datafile in files:
    xls = pd.ExcelFile(datafile)
    core_names = xls.sheet_names[:]
    for i,lab in enumerate(core_names):
        df = pd.read_excel(datafile,sheet_name=i,skiprows = 0, header = None).replace('n.a.', np.nan)
        
        ## todo later: adjust the code for then sample_date is not defined
        #sample_date = datetime.date(2000,1,1) # just a random date for now
        
        core_depth = df.to_numpy()[1:,0].astype(None)
        
        if not lab in new_dic['tiepoints'].keys():
            #initialize tiepoints with empty lists
            new_dic['tiepoints'][lab] = []
            
        if not lab in new_dic['cores'].keys():
            new_dic['cores'][lab] = {}      
        
        for i in range(1,len(df.T)):
            chem_name = df.to_numpy()[0,i]
            chem_profile = df.to_numpy()[1:,i].astype(None)
            
            # for Agnese in july 2024: restrict to the first 18 meters
            ind = (core_depth<18)
            
            new_dic['cores'][lab][chem_name] = {}
            new_dic['cores'][lab][chem_name]['data'] = chem_profile[ind].copy()
            new_dic['cores'][lab][chem_name]['depth'] = core_depth[ind].copy()

In [39]:
datafolder = './data/metadata/'
p = Path(datafolder)  
files = sorted(list(p.glob(f'*.xlsx')))
for datafile in files:
    xls = pd.ExcelFile(datafile)
    core_names = xls.sheet_names[:]
    for i,lab in enumerate(core_names):
        df = pd.read_excel(datafile,sheet_name=i,skiprows = 0, header = None).replace('n.a.', np.nan)
        if not lab in new_dic['metadata'].keys():
            #initialize tiepoints with empty lists
            new_dic['metadata'][lab] = {}
        for i in range(0,len(df.T)):
            #metadata_name = df.to_numpy()[0,i]
            metadata_name = df.iloc[0,i]

            #metadata_value = df.to_numpy()[1,i]
            metadata_value = df.iloc[1,i]

            new_dic['metadata'][lab][metadata_name] = metadata_value

In [40]:
ref_lab = 'IGE48'

In [41]:
new_dic['cores']['REF'] = new_dic['cores'][ref_lab].copy()
new_dic['metadata']['REF'] = new_dic['metadata'][ref_lab].copy()

In [42]:
with open('aligv2_agnese1.pkl','wb') as fp:
    pickle.dump(new_dic,fp)

## new completed data_file
datafile = './data/agnese_chem_all.xlsx'

# we've compiled all of Joel's stuff in a single file
xls = pd.ExcelFile(datafile)

# each sheet corresponds to a pit

core_names = xls.sheet_names[:]

for i,lab in enumerate(core_names):
    
    df = pd.read_excel(datafile,sheet_name=i,skiprows = 0, header = None).replace('n.a.', np.nan)
    
    #sample_date_timestamp = df.iloc[0,3]
    #sample_date = sample_date_timestamp.date()
    sample_date = datetime.date(2000,1,1) # just a random date for now
    
    core_depth = df.to_numpy()[1:,0].astype(None)
    
    #print(core_depth)
    
    new_dic['cores'][lab] = {}
    
    new_dic['cores'][lab]['time'] = sample_date
    
    new_dic['cores'][lab]['depth'] = core_depth.copy()
    new_dic['cores'][lab]['data'] = {}
    
    # initialize tiepoints with empty lists
    
    new_dic['tiepoints'][lab] = {'xp1':[],'xp2':[]}
    
    for i in range(1,len(df.T)):
        chem_name = df.to_numpy()[0,i]
        chem_profile = df.to_numpy()[1:,i].astype(None)
        new_dic['cores'][lab]['data'][chem_name] = chem_profile.copy()

fig,ax = plt.subplots(figsize=(10,8))
for key,value in new_dic['PROFILES'].items():
    plt.plot(value['depth'],value['chemistry']['SO4'],label=key)
plt.legend()

plt.plot(new_dic['REF']['depth'],new_dic['REF']['chemistry']['SO4'],label=key,color='red',lw=5)

# REUSE POINTS FROM OLD DATASTRUCTURE ALIGNMENT

In [55]:
old_dic = load_dic_file('alig_agnese5A.pkl') # insert name of old alignment file here

In [56]:
new_tiepoints = {}

In [57]:
for key in ['LSCE18', 'VE18', 'VE48', 'IGE48']:
    new_tiepoints[key] = []
    xp1 = old_dic[key]['xp1']
    xp2 = old_dic[key]['xp2']
    
    for i in range(len(xp2)):
        new_tp = {'species' : None,
                 'ref_depth' : xp2[i],
                 'profile_depth' : xp1[i]}
        new_tiepoints[key].append(new_tp)

In [58]:
new_dic2 = new_dic.copy()

In [60]:
new_dic2['tiepoints'] = new_tiepoints.copy()

In [61]:
write_dic_file(new_dic2,'aligv2_agnese5A.pkl')

# REUSE POINTS FROM PREVIOUS ALIGNMENT:

In [ ]:
#xp1_dic,xp2_dic = load_marked_points('#########.pkl') # insert name of old alignment file here

In [ ]:
#for key in xp1_dic.keys():
#    new_dic['tiepoints'][key] = {}
#    new_dic['tiepoints'][key]['xp1'] = xp1_dic[key]
#    new_dic['tiepoints'][key]['xp2'] = xp2_dic[key]

In [ ]:
#with open('alig_agnese4.pkl','wb') as fp:
#    pickle.dump(new_dic,fp)